# Урок 7. Сверточные нейронные сети для анализа текста

## Домашнее задание

Берем отызывы за лето (из архива с материалами или предыдущего занятия)
1. Учим conv сеть для классификации
2. Рассмотреть 2-а варианта сеточек
2.1 Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
2.2 Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)

Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше

### Подключение библиотек

In [2]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32911 sha256=431a07dc642c795e3caef60300cd88f2f7c95e1b5f49c16002f3be30e2c2d349
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [3]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 8.2 MB 45.5 MB/s 


In [34]:
import numpy as np
import pandas as pd

from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

from sklearn.model_selection import train_test_split

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
from keras.callbacks import EarlyStopping  
from keras.utils import np_utils

### Загрузка и подготовка данных

In [5]:
df = pd.read_excel('/content/drive/MyDrive/отзывы за лето.xlsx')
df.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


Предобработка текста

In [7]:
sw = get_stop_words('ru')
punct = set(punctuation)
morpher = MorphAnalyzer()

In [16]:
def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in punct)
    txt = txt.lower()
    txt = re.sub("не\s", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [17]:
df['Content'] = df['Content'].apply(preprocess_text)

In [18]:
df['Content'].head()

0                                        it just works
1    целое удобноной приложениеиз минус хотеть боль...
2                                              отлично
3    зависать 1 работа антивирус ранее пользоваться...
4                               удобно работать быстро
Name: Content, dtype: object

Разделение выборки на тренировочную, тестовую и валидационную

In [20]:
train_df, test_val_df = train_test_split(df, test_size=0.3, random_state=17)
test_df, val_df = train_test_split(test_val_df, test_size=0.5, random_state=17)

Создание корпуса

In [21]:
train_corpus = " ".join(train_df['Content'])
train_corpus = train_corpus.lower()

Токенизация корпуса

In [23]:
nltk.download("punkt")
tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Сортировка и отбор самых частотных токенов

In [29]:
# Настраиваемые параметры сети
max_words = 1000
max_len = 100
num_classes = 6

In [30]:
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [31]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [32]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [33]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in train_df['Content']], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in test_df['Content']], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in val_df['Content']], dtype=np.int32)

### Построение нейронной сети

In [35]:
y_train = keras.utils.np_utils.to_categorical(train_df['Rating'], num_classes)
y_test = keras.utils.np_utils.to_categorical(test_df['Rating'], num_classes)
y_val = keras.utils.np_utils.to_categorical(val_df['Rating'], num_classes)

In [39]:
# Настраиваемые параметры обучения
epochs = 20
batch_size = 512
print_batch_n = 100

In [36]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [37]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [40]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 11s 368ms/step - loss: 1.2897 - accuracy: 0.6781 - val_loss: 1.0841 - val_accuracy: 0.6814
Epoch 2/20
26/26 [==============================] - 9s 365ms/step - loss: 0.9167 - accuracy: 0.7093 - val_loss: 0.8715 - val_accuracy: 0.7070


In [41]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 1s 85ms/step - loss: 0.8015 - accuracy: 0.7325


Test score: 0.8015370965003967
Test accuracy: 0.7324943542480469


In [42]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 1s 91ms/step


### Embedding слой с предобученными векторами

Загрузка предобученной модели

In [43]:
!wget https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz

!gunzip taiga_upos_skipgram_300_2_2018.vec.gz

--2022-07-22 06:31:10--  https://rusvectores.org/static/models/rusvectores4/taiga/taiga_upos_skipgram_300_2_2018.vec.gz
Resolving rusvectores.org (rusvectores.org)... 116.203.104.23
Connecting to rusvectores.org (rusvectores.org)|116.203.104.23|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346385366 (330M) [application/x-gzip]
Saving to: ‘taiga_upos_skipgram_300_2_2018.vec.gz’

taiga_upos_skipgram 100%[===================>] 330.34M  30.0MB/s    in 12s     

2022-07-22 06:31:22 (28.3 MB/s) - ‘taiga_upos_skipgram_300_2_2018.vec.gz’ saved [346385366/346385366]



In [44]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_df['Content'])
vocab_size = len(tokenizer.word_index) + 1

In [45]:
embeddings_index = {}
with open('taiga_upos_skipgram_300_2_2018.vec') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        word = word.split('_')[0]
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

Found 209666 word vectors.


Матрица эмбеддингов

In [46]:
embedding_dim = 300
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 4886 words (4993 misses)


In [47]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=300, input_length=max_len, weights=[embedding_matrix],
    trainable=False))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [48]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [49]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
26/26 [==============================] - 16s 596ms/step - loss: 1.4294 - accuracy: 0.6741 - val_loss: 1.1546 - val_accuracy: 0.6807
Epoch 2/20
26/26 [==============================] - 14s 552ms/step - loss: 0.9632 - accuracy: 0.7326 - val_loss: 0.9026 - val_accuracy: 0.7326


In [50]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

7/7 [==============================] - 1s 187ms/step - loss: 0.8352 - accuracy: 0.7619


Test score: 0.8351670503616333
Test accuracy: 0.7618586421012878


In [51]:
results = model.predict(x_test, batch_size=batch_size, verbose=1)

7/7 [==============================] - 2s 246ms/step
